In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop, SGD, Adagrad, Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [3]:
df = pd.read_csv('dataset/grid_taxi_16times_grid_path.txt',delimiter=',',encoding='latin-1',header=-1)

In [4]:
df[0]

0                           W 20
1                           W 19
2                           N 19
3                           E 17
4                      W 16 S 17
5                N 14 S 18 NW 20
6                           E 17
7                           N 14
8                      S 19 N 20
9                          SW 18
10                         SE 20
11                          S 15
12                          S 15
13                     N 17 S 20
14      W 14 S 15 N 16 S 17 N 20
15                          E 14
16                     E 14 N 19
17                     N 14 W 19
18                          S 17
19                          E 19
20                     S 14 N 19
21                     N 17 S 18
22                S 17 E 19 W 20
23                          S 17
24                     N 17 S 19
25                          W 20
26                          W 15
27                     W 14 E 16
28                     W 15 E 18
29                E 14 W 15 E 16
          

In [5]:
df[1] = df[0].str[-5:]

In [6]:
df.head(10)

,0,1
0,W 20,W 20
1,W 19,W 19
2,N 19,N 19
3,E 17,E 17
4,W 16 S 17,S 17
5,N 14 S 18 NW 20,NW 20
6,E 17,E 17
7,N 14,N 14
8,S 19 N 20,N 20
9,SW 18,SW 18


In [7]:
df[1] = df[1].str[:2]

In [8]:
df[0] = df[0].str[:-5]

In [9]:
df.head(10)

,0,1
0,,W
1,,W
2,,N
3,,E
4,W 16,S
5,N 14 S 18,NW
6,,E
7,,N
8,S 19,N
9,,SW


In [10]:
df[2] = df[0]

In [11]:
df[2].replace('', np.nan, inplace=True)

In [12]:
df

,0,1,2
0,,W,NaN
1,,W,NaN
2,,N,NaN
3,,E,NaN
4,W 16,S,W 16
5,N 14 S 18,NW,N 14 S 18
6,,E,NaN
7,,N,NaN
8,S 19,N,S 19
9,,SW,NaN


In [14]:
df.dropna(subset=[2], inplace=True)

In [16]:
#test = df

In [17]:
#test = test[0] + test[1]

In [20]:
df[1] = df[1].str.strip()

In [21]:
df.head(10)

,0,1,2
4,W 16,S,W 16
5,N 14 S 18,NW,N 14 S 18
8,S 19,N,S 19
13,N 17,S,N 17
14,W 14 S 15 N 16 S 17,N,W 14 S 15 N 16 S 17
16,E 14,N,E 14
17,N 14,W,N 14
20,S 14,N,S 14
21,N 17,S,N 17
22,S 17 E 19,W,S 17 E 19


In [43]:
test_1 = df[1]
test_1 = le.fit_transform(test_1)
test_1.shape

(1355, 8)

In [22]:
X = df[0]
Y = df[1]

In [23]:
le = LabelBinarizer()
Y = le.fit_transform(Y)
Y.reshape(-1,8)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
Y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [25]:
X.shape

(1355,)

In [26]:
Y.shape

(1355, 8)

In [27]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [28]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [44]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(8,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [66]:
model = RNN()
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=Adagrad(),metrics=['accuracy'])

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 150, 50)           50000     
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_19 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 8)                 205

In [67]:
model.fit(sequences_matrix,Y_train,batch_size=1,epochs=10,
          validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.001)])

C:\Users\Hyojun\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1035 samples, validate on 116 samples
Epoch 1/10
1035/1035 [==============================] - 44s 43ms/step - loss: 1.3445 - accuracy: 0.5478 - val_loss: 0.9439 - val_accuracy: 0.7500
Epoch 2/10
1035/1035 [==============================] - 47s 46ms/step - loss: 1.0093 - accuracy: 0.7198 - val_loss: 0.8866 - val_accuracy: 0.7586
Epoch 3/10
1035/1035 [==============================] - 46s 44ms/step - loss: 0.9691 - accuracy: 0.7314 - val_loss: 0.8793 - val_accuracy: 0.7500
Epoch 4/10
1035/1035 [==============================] - 45s 44ms/step - loss: 0.9428 - accuracy: 0.7304 - val_loss: 0.8547 - val_accuracy: 0.7672s: 0.948
Epoch 5/10
1035/1035 [==============================] - 45s 44ms/step - loss: 0.9336 - accuracy: 0.7314 - val_loss: 0.8446 - val_accuracy: 0.7672
Epoch 6/10
1035/1035 [==============================] - 45s 44ms/step - loss: 0.9356 - accuracy: 0.7333 - val_loss: 0.8372 - val_accuracy: 0.7672
Epoch 7/10
1035/1035 [==============================] - 45s 43ms/step

In [68]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [69]:
accr = model.evaluate(test_sequences_matrix,Y_test)

204/204 [==============================] - 0s 515us/step


In [70]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.847
  Accuracy: 0.735
